In [6]:
import torch
from torch_geometric.data import Dataset
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import Linear
from torch_geometric.nn.dense import DenseGCNConv, DMoNPooling
import matplotlib.pyplot as plt
import time
from torch_geometric.utils import to_dense_batch
from torch_geometric.utils import to_dense_adj


torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MNIST_PATH = "../datasets/MNISTSuperpixel"

dataset: Dataset = MNISTSuperpixels(root=MNIST_PATH).shuffle()
test_dataset: Dataset = MNISTSuperpixels(root=MNIST_PATH, train=True)
train_loader = DataLoader(dataset[:2500], batch_size=128, shuffle=True)
test_loader = DataLoader(dataset, batch_size=128)

In [11]:
class GCN_Dense(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = DenseGCNConv(in_channels, hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels)
        # self.conv3 = DenseGCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, out_channels)
        

    def forward(self, x, adj, mask):
        x = self.conv1(x, adj, mask).relu()
        x = self.conv2(x, adj, mask).relu()
        # x = self.conv3(x, adj, mask).relu()
        x = x.sum(dim=1)
        return self.lin(x)

In [12]:
model = GCN_Dense(
    in_channels=dataset.num_features,
    hidden_channels=8,
    out_channels=dataset.num_classes,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        x, mask = to_dense_batch(data.x, data.batch)
        adj = to_dense_adj(data.edge_index, data.batch)
        optimizer.zero_grad()
        out = model(x, adj, mask)
        loss = F.cross_entropy(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    total_correct = 0
    for data in loader:
        data = data.to(device)
        x, mask = to_dense_batch(data.x, data.batch)
        adj = to_dense_adj(data.edge_index, data.batch)
        out = model(x, adj, mask)
        pred = out.argmax(dim=-1)
        total_correct += int((pred == data.y).sum())
    return total_correct / len(loader.dataset)

In [13]:
times = []
for epoch in range(1, 101):
    start = time.time()
    loss = train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    times.append(time.time() - start)
    print(f'\rEpoch: {epoch:03d}, Loss: {loss:.4f}, Train_acc: {train_acc:.4f}, '
          f'Test_acc: {test_acc:.4f}', end="")
print(f'\nMedian time per epoch: {torch.tensor(times).median():.4f}s')

Epoch: 005, Loss: 2.2462, Train_acc: 0.1448, Test_acc: 0.1293

KeyboardInterrupt: 